Custom Workloads with Futures
=============================

<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg" 
     width="30%" 
     align=right
     alt="Dask logo">

Dask futures provide fine-grained real-time execution for custom situations.  This is the foundation for other APIs like Dask arrays and dataframes.

## Start Dask Client

Unlike for arrays and dataframes, you need the Dask client to use the Futures interface.  Additionally the client provides a dashboard which 
is useful to gain insight on the computation.

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4, n_workers=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57644,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57649,Total threads: 4
Dashboard: http://127.0.0.1:57650/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:57647,


## Create simple functions

These functions do simple operations like add two numbers together, but they sleep for a random amount of time to simulate real work.

In [2]:
import time
import random

def inc(x):
    time.sleep(random.random())
    return x + 1

def double(x):
    time.sleep(random.random())
    return 2 * x
    
def add(x, y):
    time.sleep(random.random())
    return x + y 

We can run them locally

In [3]:
inc(1)

2

Or we can submit them to run remotely with Dask.  This immediately returns a future that points to the ongoing computation, and eventually to the stored result.

In [4]:
future = client.submit(inc, 1)  # returns immediately with pending future
future

<Future: pending, key: inc-5c519cebf96e7359ffc1f8a51fcc3d2c>

If you wait a second, and then check on the future again, you'll see that it has finished.

In [5]:
future  # scheduler and client talk constantly

<Future: finished, type: int, key: inc-5c519cebf96e7359ffc1f8a51fcc3d2c>

You can block on the computation and gather the result with the `.result()` method.

In [6]:
future.result()

2

## Chain dependencies

You can submit tasks on other futures.  This will create a dependency between the inputs and outputs.  Dask will track the execution of all tasks, ensuring that downstream tasks are run at the proper time and place and with the proper data.

In [7]:
x = client.submit(inc, 1)
y = client.submit(double, 2)
z = client.submit(add, x, y)
z

<Future: pending, key: add-0229c53dc936a41a1aace7bcf8d38bb1>

In [8]:
z.result()

6

Note that we never blocked on `x` or `y` nor did we ever have to move their data back to our notebook.

## Submit many tasks

So we've learned how to run Python functions remotely.  This becomes useful when we add two things:

1.  We can submit thousands of tasks per second
2.  Tasks can depend on each other by consuming futures as inputs

We submit many tasks that depend on each other in a normal Python for loop

In [9]:
zs = []

In [10]:
%%time

for i in range(256):
    x = client.submit(inc, i)     # x = inc(i)
    y = client.submit(double, x)  # y = inc(x)
    z = client.submit(add, x, y)  # z = inc(y)
    zs.append(z)

CPU times: user 933 ms, sys: 51.8 ms, total: 985 ms
Wall time: 971 ms


In [11]:
total = client.submit(sum, zs)

In [13]:
zs[255].result()

768

`zs[0]` is the result of the 0th iteration of `z = x + y` where `x` is 0+1 and `y` is 1*2

In [14]:
total

<Future: finished, type: int, key: sum-bad157e18b493c3ea5d891649d3416ea>

To make this go faster, add an additional workers with more cores 

(although we're still only working on our local machine, this is more practical when using an actual cluster)

In [29]:
client.cluster.scale(8)  # ask for ten 4-thread workers

## Custom computation: Tree summation

As an example of a non-trivial algorithm, consider the classic tree reduction.  We accomplish this with a nested for loop and a bit of normal Python logic.

```
finish           total             single output
    ^          /        \
    |        c1          c2        neighbors merge
    |       /  \        /  \
    |     b1    b2    b3    b4     neighbors merge
    ^    / \   / \   / \   / \
start   a1 a2 a3 a4 a5 a6 a7 a8    many inputs
```

In [15]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = client.submit(add, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                   # swap old list for new

`L` is now a list containing futures.

If you're watching the [dashboard's status page](../proxy/8787/status) then you may want to note two things:

1.  The red bars are for inter-worker communication.  They happen as different workers need to combine their intermediate values
2.  There is lots of parallelism at the beginning but less towards the end as we reach the top of the tree where there is less work to do.

Alternatively you may want to navigate to the [dashboard's graph page](../proxy/8787/graph) and then run the cell above again.  You will be able to see the task graph evolve during the computation.

Building a computation dynamically
----------------------------------

In the examples above we explicitly specify the task graph ahead of time.  We know for example that the first two futures in the list `L` will be added together.  

Sometimes this isn't always best though, sometimes you want to dynamically define a computation as it is happening.  For example we might want to sum up these values based on whichever futures show up first, rather than the order in which they were placed in the list to start with.

For this, we can use operations like [as_completed](http://dask.pydata.org/en/latest/futures.html#distributed.as_completed).

We recommend watching the dashboard's graph page when running this computation.  You should see the graph construct itself during execution.

`as_completed` returns an iterator containing the futures in the order that they're completed. Calling `next` on the operator will block until the next future completes, irrespective of order.

You can also add more futures to this object during computation using the `.add` method.

In [37]:
del future, L, new_L, total  # clear out some old work

`zs` is a list of futures, each containing the result of `z = x + y`.

Passing this list of futures into the `inc` function means we're submitting futures as input. Each future will be incremented by 1.

In [27]:
from dask.distributed import as_completed

zs = client.map(inc, zs)
seq = as_completed(zs)

# for future in seq:
#     print(seq.next().result())
    
# is the same as
for future in seq:
    print(future.result())

11
14
8
23
17
5
32
20
26
41
35
44
50
53
56
29
38
47
68
59
65
74
62
77
83
71
80
86
89
92
95
98
101
113
107
104
116
119
110
131
122
134
125
137
128
140
149
146
143
155
158
152
161
173
170
167
182
164
176
179
185
188
200
197
191
194
209
203
215
206
221
212
218
230
224
227
239
245
233
236
254
248
242
260
251
263
257
266
269
275
272
278
281
284
293
296
287
290
308
305
311
299
302
317
314
329
332
326
320
335
341
323
344
338
350
356
347
353
368
371
359
362
377
365
374
386
383
389
380
401
392
407
395
398
404
419
413
416
422
410
425
431
440
434
428
449
437
455
443
446
452
461
467
458
476
464
470
485
488
479
473
491
482
497
503
494
509
500
506
515
524
518
512
521
527
536
530
542
533
545
554
551
548
539
563
569
557
560
566
572
578
587
575
590
584
581
593
599
596
608
602
605
620
623
611
614
617
626
635
641
629
644
632
638
647
650
656
653
668
662
674
659
677
665
683
680
689
695
686
671
692
698
701
704
713
719
710
707
728
722
716
734
731
725
740
737
746
743
749
752
761
755
764
758
767
770


NOTE: use `seq` to iterate over futures as they're completed (dynamic computation building). `seq` does not hold the results, those are in `zs`

In [28]:
while seq.count() > 1:  # at least two futures left
    a = next(seq)
    b = next(seq)
    new = client.submit(add, a, b)  # add them together 
    seq.add(new)                    # add new future back into loop

We can use `.add()` to dynamically add computations into the task graph as it's running